# Generation of professional movements with VAE-RGOM

In this Jupyter Notebook, we will see the modeling of human movements using a Variational Autoencoder (VAE).
VAE computes the time-varying coefficients of the full-body GOM representations, which are then used to generate professional movements.

The script is divided into six sections:

**1.** Loading of the libraries and pickles files containing the joint angles of seven datasets and their labels 

**2.** Obtaining the indexes for the train set and test set using stratified cross-validation

**3.** The dataset merged from all seven datasets is preprocessed

**4.** Define the structure of the VAE-RGOM network

**5.** Training and validation of VAE-RGOM using the training and testing indexes off the 5-fold cross-validation

**6.** Results of the testing tests for each of the seven datasets

**Note that the full description of this deep state-space model is provided in** [(1)](https://arxiv.org/abs/2304.14502)
    

###  1. Load the libraries and motion data of the seven datasets with professional tasks:



In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import pickle 
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv1D, Layer, LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, \
    multiply, concatenate, Flatten, Activation, dot, Lambda, Reshape, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle
import math
from sklearn.metrics import mean_squared_error, pairwise_distances, mean_absolute_error
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from scipy.spatial.distance import cdist, euclidean

# The local joint angles of each dataset are saved in pickle files. 
# Each pickle file contains the data frames of the movement iterations, and their paths are indicated in the file_list.

with open('ERGD_EulerLocalAngles.pkl', 'rb') as f: [data_ERGD, file_list_ERGD, usedJoints] = pickle.load(f)
with open('APA_EulerLocalAngles.pkl', 'rb') as f: [data_APA, file_list_APA, _] = pickle.load(f)
with open('TVA_EulerLocalAngles.pkl', 'rb') as f: [data_TVA, file_list_TVA, _] = pickle.load(f)
with open('TVP_EulerLocalAngles.pkl', 'rb') as f: [data_TVP, file_list_TVP, _] = pickle.load(f)
with open('SLW_EulerLocalAngles.pkl', 'rb') as f: [data_SLW, file_list_SLW, _] = pickle.load(f)
with open('GLB_EulerLocalAngles.pkl', 'rb') as f: [data_GLB, file_list_GLB, _] = pickle.load(f)
with open('MSC_EulerLocalAngles.pkl', 'rb') as f: [data_MSC, file_list_MSC, _] = pickle.load(f)

angles_labels = data_ERGD[0].columns

# All datasets are merged into one, and their corresponding labels are indicated in the list "labels" from the pickle file.

data = data_ERGD+data_APA+data_TVA+data_TVP+data_SLW+data_GLB+data_MSC

file_list = np.concatenate((file_list_ERGD, file_list_APA,file_list_TVA,file_list_TVP,file_list_SLW,file_list_GLB,file_list_MSC))

with open('labels_allFiles_set.pkl', 'rb') as f: [labels, _] = pickle.load(f)

###  2. Obtain the indexes of the train set and test set. 

As there are more iterations in some professional tasks than others, stratified cross-validation is applied to evaluate the model's performance.

**Note** `random_state=1` in order to obtain the same indexes and for future comparing the performance of other models in the generation of the professional movements.


In [2]:
# 5-FOLD 
train_set = []
test_set = []
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state=1)

for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
    train_set.append(train_index)
    test_set.append(test_index)

C:\Users\aimove\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


### 3. Preprocessing of the data for VAE-RGOM

In this cell, the data is normalized and divided by overlapping windows of three seconds. Then, these windows are processed by the network for making one-step predictions.


In [3]:
scalers = {} # The scalers for each joint angle are saved for the future inverse transformation
dataX =[]  # The overlapping three-second windows are placed in dataX
dataY =[] # The prediction of the corresponding windows is placed in dataY

for dd in range(0,len(data)):
    dat = data[dd].values[1:,:]  
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(dat)
    scalers[dd] = scaler
    ned = scaler.transform(dat)
        
    wx = []
    wy = []
    for w in range(0,len(dat)-3, 1):
        wx.append(np.arange(0+w,3+w))
        wy.append(np.arange(3+w,4+w))
    
    dX=[]
    dY = []
    for wi in range(0,len(wx)):
        dX.append(ned[wx[wi],:])
        dY.append(ned[wy[wi],:])
    dataX.append(np.array(dX))
    dataY.append(np.array(dY))

### 4. VAE-RGOM

**4.1** Next is defined the VAE-RGOM class and functions used for the one-step prediction.

In [4]:

# Hadmard product is applied to solve the GOM representations given the tensor of coefficients provided by VAE
def hadamard_product(x):
    coef = x[0]
    inp = x[1]
    inp2 = K.expand_dims(inp, axis=1)
    m1 = coef*inp2
    m2 = K.sum(m1, axis=(2,3)) 
    y = K.expand_dims(m2, axis=-2)
    return y

# Sampling from the latent distribution created by the encoder
class Sampling(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
    
# Creation of class for VAE-RGOM:
class VAE_RGOM(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE_RGOM, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.GOM_loss_MSE_tracker = keras.metrics.Mean(name="MSE_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.MAE_loss_tracker = keras.metrics.Mean(name="MAE_loss")
        self.val_loss_tracker = keras.metrics.Mean(name="val_loss")
        self.valMAE_loss_tracker = keras.metrics.Mean(name="val_MAE_loss")
        
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.GOM_loss_MSE_tracker,
            self.kl_loss_tracker,
            self.MAE_loss_tracker
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            dataInput, y_true  = data
            z_mean, z_log_var, z, encoder_last_h, encoder_last_c = self.encoder(dataInput)
            y_pred = self.decoder([z, dataInput, encoder_last_h, encoder_last_c])
            
            mse = tf.keras.losses.MeanSquaredError(reduction=keras.losses.Reduction.SUM)
            mae = tf.keras.losses.MeanAbsoluteError()
            
            loss_GOM_MSE = tf.reduce_mean(mse(y_true, y_pred)) # Prediction loss as the mean squared error
            loss_MAE = tf.reduce_mean(mae(y_true, y_pred)) # Observation of the mean absolute error
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)) # KL divergence loss
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        
                        
            total_loss = kl_loss + loss_GOM_MSE # The total loss of VAE-RGOM corresponds to the prediction loss and KL divergence loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.GOM_loss_MSE_tracker.update_state(loss_GOM_MSE)
        self.kl_loss_tracker.update_state(kl_loss)
        self.MAE_loss_tracker.update_state(loss_MAE)
        return {
            "loss": self.total_loss_tracker.result(),
            "MSE_loss": self.GOM_loss_MSE_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "MAE_loss": self.MAE_loss_tracker.result(),
        }
    def test_step(self, data):
        dataInput, y_true   = data
        _, _, z, encoder_last_h, encoder_last_c = self.encoder(dataInput)
        y_pred = self.decoder([z, dataInput, encoder_last_h, encoder_last_c])
        mse = tf.keras.losses.MeanSquaredError(reduction=keras.losses.Reduction.SUM)
        mae = tf.keras.losses.MeanAbsoluteError()
        
        val_loss = tf.reduce_mean(mse(y_true, y_pred))
        mae_loss = tf.reduce_mean(mae(y_true, y_pred))

        
        self.val_loss_tracker.update_state(val_loss)
        self.valMAE_loss_tracker.update_state(mae_loss)
        
        return {"loss": self.val_loss_tracker.result(),
                "MAE_loss": self.valMAE_loss_tracker.result(),}
    
    def predict(self, dataInput):
        _, _, z, encoder_last_h, encoder_last_c = self.encoder(dataInput)
        y_pred = self.decoder([z, dataInput, encoder_last_h, encoder_last_c])
        return y_pred

**4.2** Following is defined the structure of the encoder and decoder of VAE-RGOM. The hyperparameters were previously selected using a Bayesian optimization.

In [5]:
latent_dim = 2
w = 3
n_hidden1 = 32
hp_act1 = 'softsign' 
hp_act2 = 'softsign' 

# ENCODER
input_train = Input(shape=(dataX[0].shape[1], dataX[0].shape[2]))
output_train = Input(shape=(dataY[0].shape[1], dataY[0].shape[2]))

encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(n_hidden1, activation= hp_act1, dropout=0.2, recurrent_dropout=0.2, 
return_state=True, return_sequences=True)(input_train)

z_mean = Dense(latent_dim, name="z_mean")(encoder_last_h)
z_log_var = Dense(latent_dim, name="z_log_var")(encoder_last_h)
z = Sampling()([z_mean, z_log_var])

encoder = Model(input_train, [z_mean, z_log_var, z, encoder_last_h, encoder_last_c], name="encoder")
encoder.summary()


# DECODER
latent_inputs = Input(shape=(latent_dim,))
dat_inputs = Input(shape=(dataX[0].shape[1], dataX[0].shape[2]))
enc_last_h = Input(shape=(n_hidden1,))
enc_last_c = Input(shape=(n_hidden1,))

decoder_input = RepeatVector(w)(latent_inputs)
decoder_stack_h = LSTM(n_hidden1, activation=hp_act2, dropout=0.2, recurrent_dropout=0.2, return_state=False, return_sequences=True)(decoder_input, initial_state=[enc_last_h, 
                                                                  enc_last_c])
decoder_stack_h = Dropout(0.8, name='dropout_1')(decoder_stack_h)
xC = TimeDistributed(Dense(dataY[0].shape[2]*dataY[0].shape[2]))(decoder_stack_h)

# Solve GOM
xC_input = Reshape((dataY[0].shape[2], w, dataY[0].shape[2]))(xC)
decoder_outputs = Lambda(hadamard_product, output_shape=(dataY[0].shape[2],))([xC_input, dat_inputs])

decoder = Model([latent_inputs, dat_inputs, enc_last_h, enc_last_c], decoder_outputs, name="decoder")
decoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 57)]      0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 3, 32), (Non 11520       input_1[0][0]                    
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 2)            66          lstm[0][1]                       
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 2)            66          lstm[0][1]                       
____________________________________________________________________________________________

### 5. Evaluation  of the model using a stratified 5-fold cross-validation
The train and test indexes define each iteration's train set and test set. Then, save the results from analyzing the performance achieved with each professional movement.

In [ ]:
kResults = []
for k in range(0, len(train_set)):
    print(f"Fold {k}:")
    train_index = train_set[k] # Obtain the train indexes for the iteration k
    test_index = test_set[k] # Obtain the test indexes for the iteration k
    X_train_cv = [dataX[tt] for tt in train_index]
    y_train_cv = [dataY[tt] for tt in train_index]
    
    X_test_cv = [dataX[tt] for tt in test_index]
    y_test_cv= [dataY[tt] for tt in test_index]
    scaler_Test = [scalers[tt] for tt in test_index]
    file_list_Test = [file_list[tt] for tt in test_index]
    lables_Test = [labels[tt] for tt in test_index]
    
    X_input_train = np.concatenate(X_train_cv, axis=0)
    X_output_train = np.concatenate(y_train_cv, axis=0)
    
    X_input_train, X_output_train = shuffle(X_input_train, X_output_train)
    
    vae = VAE_RGOM(encoder, decoder) # Create VAE-RGOM
    opt = Adam(learning_rate=0.0001, clipnorm=1.0)
    vae.compile(optimizer=opt, run_eagerly=True)
    
    epc = 50
    es = EarlyStopping(monitor='val_loss', mode='min', patience=2)
    history = vae.fit(X_input_train, X_output_train, validation_split=0.1,  
                        epochs=epc, verbose=1, callbacks=[es], 
                        batch_size=1024) # Training of the model
    
    # Evaluate the performance of the trained model with the test set
    metrics = pd.DataFrame() # Save results in the metrics data frame
    for f in range(0, len(X_test_cv)):
        df_test = X_test_cv[f]
        y = y_test_cv[f]
        scaler_T = scaler_Test[f]
        
        y=y.reshape([y.shape[0],y.shape[2]])
        
        z_mean, z_log_var, z, encoder_last_h, encoder_last_c = vae.encoder(df_test)
        d1_pred = vae.decoder([z, df_test, encoder_last_h, encoder_last_c])
        pred_VAE = keras.backend.get_value(d1_pred)
        
        pred_VAE=pred_VAE.reshape([pred_VAE.shape[0],pred_VAE.shape[2]])
        
        pred_VAE = scaler_T.inverse_transform(pred_VAE)
        y = scaler_T.inverse_transform(y)
        
        # Metrics used for the evaluation: 
        mse_t = mean_squared_error(y, pred_VAE) # Mean Squared Error
        mae_t = mean_absolute_error(y, pred_VAE) # Mean Absolute Error
        pde_t = [cdist(y[:,ii].reshape(1,-1), pred_VAE[:,ii].reshape(1,-1), 'euclidean') for ii in range(0,y.shape[1])] # Average Pairwise Distance
        fde_t = [euclidean(y[-1,ii], pred_VAE[-1,ii]) for ii in range(0,y.shape[1])] # Final Displacement Error
        
        met_f = pd.Series({
                'File' : file_list_Test[f],
                'Class' : lables_Test[f],
                'MSE' : mse_t,
                'MAE' : mae_t,
                'APD' : np.mean(pde_t),
                'AFDE': np.mean(fde_t),
            })
        
        metrics = pd.concat([metrics, met_f], axis = 1)
        
    kResults.append(metrics)

In [7]:
# SAVE RESULTS AND MODEL
#with open('VAERGOM_kfold_results.pkl', 'wb') as f: pickle.dump([kResults], f)
#encoder.save('VAERGOM_encoder')
#decoder.save('VAERGOM_decoder')

# LOAD SAVED RESULTS
#with open('VAERGOM_kfold_results.pkl', 'rb') as f: [kResults] = pickle.load(f)

### 6. Metrics obtained for the movements of each dataset:

In [10]:
# ERGD
c=28
classD = np.arange(0,c) # Labels of the movements from the ERGD dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
ERGD_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
ERGD_results.describe()

,Class,MSE,MAE,APD,AFDE
count,28.000000,27.000000,27.000000,27.000000,27.000000
mean,13.500000,3.912998,0.052597,7.495478,0.065128
std,8.225975,3.597042,0.032914,5.952281,0.098867
min,0.000000,0.000669,0.002982,0.142846,0.003080
25%,6.750000,1.101289,0.023829,2.319817,0.022226
50%,13.500000,2.916668,0.052146,7.306047,0.043427
75%,20.250000,6.112824,0.073738,10.746015,0.052838
max,27.000000,14.623239,0.131636,23.146294,0.494952


In [11]:
# APA
c = c+3
classD = np.arange(c-3,c) # Labels of the movements from the APA dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
APA_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
APA_results.describe()

,Class,MSE,MAE,APD,AFDE
count,3.0,3.000000,3.000000,3.000000,3.000000
mean,29.0,1.740152,0.068327,15.453044,0.238978
std,1.0,0.606371,0.014698,9.792951,0.260619
min,28.0,1.313932,0.053190,6.309257,0.078144
25%,28.5,1.393057,0.061220,10.286331,0.088631
50%,29.0,1.472182,0.069250,14.263404,0.099118
75%,29.5,1.953262,0.075896,20.024937,0.319394
max,30.0,2.434342,0.082543,25.786471,0.539671


In [12]:
# TVA
c = c+4
classD = np.arange(c-4,c) # Labels of the movements from the TVA dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
TVA_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
TVA_results.describe()

,Class,MSE,MAE,APD,AFDE
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,32.500000,3.109723,0.070500,5.460340,0.099131
std,1.290994,2.030953,0.015753,3.430544,0.028843
min,31.000000,0.443331,0.046947,3.331939,0.076698
25%,31.750000,2.417519,0.069185,3.448597,0.082919
50%,32.500000,3.318195,0.077735,3.979625,0.089351
75%,33.250000,4.010399,0.079050,5.991368,0.105563
max,34.000000,5.359173,0.079581,10.550170,0.141122


In [13]:
# TVP
c = c+9
classD = np.arange(c-9,c) # Labels of the movements from the TVP dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
TVP_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
TVP_results.describe()

,Class,MSE,MAE,APD,AFDE
count,9.000000,9.000000,9.000000,9.000000,9.000000
mean,39.000000,13.132307,0.164899,52.373109,0.099444
std,2.738613,10.392732,0.059236,19.037514,0.026249
min,35.000000,1.934931,0.089476,25.975985,0.071597
25%,37.000000,5.324730,0.125045,36.672780,0.075244
50%,39.000000,10.005028,0.166984,53.719419,0.089676
75%,41.000000,16.259985,0.181710,71.053214,0.117931
max,43.000000,30.462696,0.264075,77.041984,0.144148


In [14]:
# SLW
c = c+16
classD = np.arange(c-16,c) # Labels of the movements from the SLW dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
SLW_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
SLW_results.describe()

,Class,MSE,MAE,APD,AFDE
count,16.000000,15.000000,15.000000,15.000000,15.000000
mean,51.500000,0.644845,0.072330,5.581857,0.087897
std,4.760952,1.353491,0.023525,8.346484,0.026541
min,44.000000,0.021445,0.048192,0.912820,0.051494
25%,47.750000,0.040674,0.054001,1.549523,0.068745
50%,51.500000,0.060563,0.071205,2.066840,0.085658
75%,55.250000,0.389229,0.077978,4.538974,0.113450
max,59.000000,4.920308,0.135351,30.108368,0.137606


In [15]:
# GLB
c = c+18
classD = np.arange(c-18,c) # Labels of the movements from the GLB dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
GLB_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
GLB_results.describe()

,Class,MSE,MAE,APD,AFDE
count,18.000000,18.000000,18.000000,18.000000,18.000000
mean,68.500000,7.630149,0.095660,13.389510,0.222670
std,5.338539,7.783253,0.043541,8.794872,0.255698
min,60.000000,0.017174,0.047310,1.165528,0.072958
25%,64.250000,3.489279,0.069024,8.498453,0.105016
50%,68.500000,4.630584,0.081508,10.343040,0.140130
75%,72.750000,9.642911,0.109051,17.701589,0.203622
max,77.000000,31.466006,0.224366,33.745466,1.132128


In [16]:
# MSC
c = c+13
classD = np.arange(c-13,c) # Labels of the movements from the MSC dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
MSC_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
MSC_results.describe()

,Class,MSE,MAE,APD,AFDE
count,13.00000,13.000000,13.000000,13.000000,13.000000
mean,84.00000,18.416822,0.187680,18.693528,0.238215
std,3.89444,19.176513,0.096230,14.135980,0.326973
min,78.00000,0.056544,0.089143,3.006290,0.063044
25%,81.00000,3.461077,0.110117,9.711228,0.087822
50%,84.00000,11.902497,0.168024,14.040603,0.099306
75%,87.00000,29.282367,0.215429,22.866751,0.137962
max,90.00000,56.148614,0.385891,53.330260,1.036649
